In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# models
clf_down = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

clf_up = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

X_train = train.drop(columns=["ret_fut","y_down","y_up"])
X_test  = test.drop(columns=["ret_fut","y_down","y_up"])

y_down_train = train["y_down"]
y_up_train   = train["y_up"]

# fit
clf_down.fit(X_train, y_down_train)
clf_up.fit(X_train, y_up_train)

# proba
test["p_down"] = clf_down.predict_proba(X_test)[:, 1]
test["p_up"]   = clf_up.predict_proba(X_test)[:, 1]

# thresholds
THR_DOWN = 0.55
THR_UP   = 0.55

# meta signal
test["mode"] = "baseline"
test.loc[test["p_down"] > THR_DOWN, "mode"] = "risk_off"
test.loc[(test["mode"] == "baseline") & (test["p_up"] > THR_UP), "mode"] = "risk_on"

# build daily weights from modes
dates_test = sorted(test.index.get_level_values(0).unique())

# weights templates
w_baseline = w_low.loc[dates_test].copy()
w_risk_on  = pd.DataFrame(1.0, index=dates_test, columns=SYMBOLS)  # equal weight
w_risk_off = pd.DataFrame(0.0, index=dates_test, columns=SYMBOLS)  # cash

# map mode per date using mean prob across assets
mode_by_date = (
    test.reset_index()
        .groupby("timestamp")[["p_down","p_up","mode"]]
        .agg({"p_down":"mean","p_up":"mean","mode":lambda x: x.iloc[0]})
)

W = pd.DataFrame(index=dates_test, columns=SYMBOLS, dtype=float)

# assign
for d in dates_test:
    m = mode_by_date.loc[d, "mode"]
    if m == "risk_off":
        W.loc[d] = w_risk_off.loc[d]
    elif m == "risk_on":
        W.loc[d] = w_risk_on.loc[d]
    else:
        W.loc[d] = w_baseline.loc[d]

# normalize
W = W.fillna(0.0)
W = W.clip(lower=0.0)
W = W.div(W.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)

# backtest util
def run_bt(weights, rets, cost_bps=10):
    w = weights.shift(1).reindex(rets.index).fillna(0.0)
    w = w.div(w.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
    gross = (w * rets).sum(axis=1)

    to = w.diff().abs().sum(axis=1).fillna(0.0)
    cost = to * (cost_bps / 10000.0)

    net = gross - cost
    return net, to

def stats(r, freq=365):
    r = r.dropna()
    eq = (1 + r).cumprod()
    mu = r.mean() * freq
    sig = r.std() * np.sqrt(freq)
    sharpe = mu / sig if sig > 0 else np.nan
    dd = (eq / eq.cummax() - 1.0).min()
    ann = eq.iloc[-1] ** (freq/len(r)) - 1 if len(r) > 0 else np.nan
    return float(ann), float(sig), float(sharpe), float(dd)

# compare on test window only
rets_test = rets.loc[dates_test]

r_meta, to_meta = run_bt(W, rets_test, cost_bps=COST_BPS)
r_low,  to_low  = run_bt(w_baseline, rets_test, cost_bps=COST_BPS)

# print
rows = []
for name, r, to in [
    ("baseline_low_vol", r_low, to_low),
    ("meta_two_models",  r_meta, to_meta),
]:
    ann, sig, sh, dd = stats(r, freq=FREQ)
    rows.append({"name": name, "ann_return": ann, "ann_vol": sig, "sharpe": sh, "max_dd": dd, "turnover": float(to.mean())})

summary = pd.DataFrame(rows).set_index("name")
print(summary)

# equity
plt.figure(figsize=(12,5))
plt.plot((1+r_low.fillna(0)).cumprod(), label="baseline_low_vol")
plt.plot((1+r_meta.fillna(0)).cumprod(), label="meta_two_models")
plt.legend()
plt.title("Test window equity")
plt.tight_layout()
plt.show()

# mode counts
print("\nmode counts")
print(mode_by_date["mode"].value_counts())


                  | ann_return  | ann_vol | sharpe | max_dd  turnover |
baseline_low_vol |-0.272841  | 0.510340 | -0.366125 | -0.449281 | 0.077413 |

meta_two_models  | -0.288073 | 0.512886 |-0.403006 | -0.464904 |  0.141166 |

![Image test 8](image/image_test_8.png)

mode counts

mode

baseline    353

risk_on      13

Name: count, dtype: int64

Sur la période test, la stratégie baseline low-vol est déjà en difficulté (rendement annualisé négatif), ce qui montre que le marché crypto était globalement défavorable sur cette fenêtre. La stratégie méta (2 modèles) ne fait pas mieux : elle obtient une performance légèrement pire et un drawdown un peu plus élevé, avec un turnover plus important.

Un point clé : le modèle choisit presque toujours le mode baseline (353 jours), très rarement risk_on (13 jours) et jamais risk_off. Donc en pratique, la stratégie “switch” très peu, et quand elle switch, ça rajoute surtout du turnover (donc des coûts) sans apporter de protection suffisante.

Ces résultats restent intéressants pour le projet : ils montrent que la pipeline ML (features → labels → train/test → signal → allocation) est bien construite, mais que la prédiction directionnelle des “gros moves” est difficile à ce stade. La suite logique serait de tester d’autres seuils, d’autres définitions de queues (ex: 10% au lieu de 30%), ou des modèles plus flexibles (arbres / boosting), tout en contrôlant le turnover.